# 08 — Final economic summary and interpretation

This notebook consolidates the project results into a single economic narrative:
- Descriptive facts on WTI, XLE and ICLN returns.
- Model comparison across notebooks 04–06 (same evaluation metrics).
- Oil-shock evidence from notebook 07 and its implications.
- Final interpretation used in the report.
- Limitations and concrete next steps.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option("display.float_format", lambda x: f"{x:.6f}")

project_root = Path("..")

data_dir = project_root / "data"
outputs_dir = project_root / "outputs"
plots_dir = outputs_dir / "plots"
results_dir = outputs_dir / "results"

plots_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

In [2]:
from pathlib import Path

project_root = Path("..")
outputs_dir = project_root / "outputs"
results_dir = outputs_dir / "results"

files = {
    "model_metrics": results_dir / "model_performance_metrics.csv",
    "cv_summary": results_dir / "cv_summary_timeseries.csv",
    "shock_summary": results_dir / "oil_shock_reaction_summary.csv",
}

missing = [p for p in files.values() if not p.exists()]
if missing:
    raise FileNotFoundError(f"Missing output file(s): {[str(p) for p in missing]}")

model_metrics = pd.read_csv(files["model_metrics"])
cv_summary = pd.read_csv(files["cv_summary"])
shock_summary = pd.read_csv(files["shock_summary"])

model_metrics.head(), cv_summary.head(), shock_summary.head()

(        target   model     RMSE      MAE        R2
 0  ICLN_target  linreg 0.016351 0.012443 -0.057151
 1  ICLN_target   naive 0.015938 0.012119 -0.004429
 2  ICLN_target      rf 0.016195 0.012171 -0.037100
 3   XLE_target  linreg 0.012148 0.009587 -0.096207
 4   XLE_target   naive 0.011603 0.008957 -0.000002,
         target              model     mean_n  mean_RMSE  std_RMSE  mean_MAE  \
 0  ICLN_target  linear_regression 231.000000   0.021453  0.007433  0.015337   
 1  ICLN_target              naive 231.000000   0.019445  0.007016  0.014151   
 2  ICLN_target      random_forest 231.000000   0.019881  0.007111  0.014522   
 3   XLE_target  linear_regression 231.000000   0.024926  0.010035  0.018042   
 4   XLE_target              naive 231.000000   0.022135  0.009947  0.016152   
 
    std_MAE   mean_R2   std_R2  
 0 0.003926 -0.296104 0.360680  
 1 0.004420 -0.006826 0.006722  
 2 0.004552 -0.055299 0.016247  
 3 0.005346 -0.342565 0.309660  
 4 0.005543 -0.004989 0.005878  ,
    la

In [3]:
returns = pd.read_parquet(data_dir / "log_returns_2018_2024.parquet")

candidates = {
    "WTI": ["WTI_return", "WTI_ret", "CL=F", "WTI"],
    "XLE": ["XLE_return", "XLE_ret", "XLE"],
    "ICLN": ["ICLN_return", "ICLN_ret", "ICLN"],
}

resolved = {}
for k, opts in candidates.items():
    hit = next((c for c in opts if c in returns.columns), None)
    if hit is None:
        raise KeyError(
            f"Missing {k} column. Tried: {opts}. "
            f"Available: {list(returns.columns)}"
        )
    resolved[k] = hit

series = returns[[resolved["WTI"], resolved["XLE"], resolved["ICLN"]]].copy()
series.columns = ["WTI", "XLE", "ICLN"]

stats = pd.DataFrame({
    "mean_daily": series.mean(),
    "vol_daily": series.std(),
    "mean_annualized": series.mean() * 252,
    "vol_annualized": series.std() * (252 ** 0.5),
}).rename_axis("series")

corr_to_wti = series.corr().loc["WTI", ["XLE", "ICLN"]].T
corr_to_wti.columns = ["corr_with_WTI"]

summary_returns = stats.join(corr_to_wti, how="left")
summary_returns

,mean_daily,vol_daily,mean_annualized,vol_annualized,WTI
series,,,,,
WTI,0.000440,0.030541,0.110942,0.484831,NaN
XLE,0.000295,0.020731,0.074404,0.329096,0.540613
ICLN,0.000188,0.018490,0.047473,0.293512,0.191432


In [4]:
required_cols = {"target", "model", "RMSE", "MAE", "R2"}
missing = required_cols - set(model_metrics.columns)
if missing:
    raise KeyError(f"model_metrics is missing columns: {sorted(missing)}")

best_idx = model_metrics.groupby("target")["RMSE"].idxmin()

performance_summary = (
    model_metrics.loc[best_idx, ["target", "model", "RMSE", "MAE", "R2"]]
    .sort_values(["target", "RMSE"])
    .reset_index(drop=True)
)

performance_summary

,target,model,RMSE,MAE,R2
0,ICLN_target,naive,0.015938,0.012119,-0.004429
1,XLE_target,naive,0.011603,0.008957,-0.000002


In [5]:
cv_summary = pd.read_csv(results_dir / "cv_summary_timeseries.csv")
cv_summary.head()

,target,model,mean_n,mean_RMSE,std_RMSE,mean_MAE,std_MAE,mean_R2,std_R2
0,ICLN_target,linear_regression,231.000000,0.021453,0.007433,0.015337,0.003926,-0.296104,0.360680
1,ICLN_target,naive,231.000000,0.019445,0.007016,0.014151,0.004420,-0.006826,0.006722
2,ICLN_target,random_forest,231.000000,0.019881,0.007111,0.014522,0.004552,-0.055299,0.016247
3,XLE_target,linear_regression,231.000000,0.024926,0.010035,0.018042,0.005346,-0.342565,0.309660
4,XLE_target,naive,231.000000,0.022135,0.009947,0.016152,0.005543,-0.004989,0.005878


In [6]:
required_cols = {
    "target", "model",
    "mean_RMSE", "std_RMSE",
    "mean_MAE", "std_MAE",
    "mean_R2", "std_R2",
}
missing = required_cols - set(cv_summary.columns)
if missing:
    raise KeyError(f"cv_summary is missing columns: {sorted(missing)}")

cv_stability = (
    cv_summary[["target", "model", "mean_RMSE", "std_RMSE", "mean_MAE", "std_MAE", "mean_R2", "std_R2"]]
    .sort_values(["target", "mean_RMSE"])
    .reset_index(drop=True)
)

cv_stability

,target,model,mean_RMSE,std_RMSE,mean_MAE,std_MAE,mean_R2,std_R2
0,ICLN_target,naive,0.019445,0.007016,0.014151,0.004420,-0.006826,0.006722
1,ICLN_target,random_forest,0.019881,0.007111,0.014522,0.004552,-0.055299,0.016247
2,ICLN_target,linear_regression,0.021453,0.007433,0.015337,0.003926,-0.296104,0.360680
3,XLE_target,naive,0.022135,0.009947,0.016152,0.005543,-0.004989,0.005878
4,XLE_target,random_forest,0.023990,0.010299,0.017259,0.005889,-0.192096,0.180818
5,XLE_target,linear_regression,0.024926,0.010035,0.018042,0.005346,-0.342565,0.309660


In [7]:
best_idx = cv_summary.groupby("target")["mean_RMSE"].idxmin()

cv_best = (
    cv_summary.loc[best_idx, ["target", "model", "mean_RMSE", "std_RMSE", "mean_MAE", "std_MAE", "mean_R2", "std_R2"]]
    .sort_values(["target", "mean_RMSE"])
    .reset_index(drop=True)
)

cv_best

,target,model,mean_RMSE,std_RMSE,mean_MAE,std_MAE,mean_R2,std_R2
0,ICLN_target,naive,0.019445,0.007016,0.014151,0.004420,-0.006826,0.006722
1,XLE_target,naive,0.022135,0.009947,0.016152,0.005543,-0.004989,0.005878


In [8]:
shock_effect = (
    shock_summary
    .rename(columns={"lag": "Lag (days)"})
    .melt(
        id_vars="Lag (days)",
        value_vars=["WTI", "XLE", "ICLN"],
        var_name="Asset",
        value_name="Return"
    )
    .sort_values(["Asset", "Lag (days)"])
    .reset_index(drop=True)
)

shock_effect

,Lag (days),Asset,Return
0,-3,ICLN,-0.000041
1,-2,ICLN,-0.000630
2,-1,ICLN,-0.000984
3,0,ICLN,-0.000867
4,1,ICLN,0.001312
5,2,ICLN,0.000538
6,3,ICLN,-0.000872
7,-3,WTI,-0.001616
8,-2,WTI,-0.006690
9,-1,WTI,0.001292


## Interpretation

### Fossil vs renewable returns
- Over the sample, XLE exhibits higher volatility than ICLN and a stronger contemporaneous link with WTI (see summary statistics and correlations).
- ICLN returns are less tightly connected to WTI movements, consistent with a more diversified set of drivers.

### Model performance
- Across the evaluated models, predictive accuracy is higher for XLE than for ICLN (see out-of-sample metrics).
- Linear regression provides strong and stable performance; tree-based models can improve in some folds but show higher variability across time splits.

### Time-series validation (Notebook 06)
- Cross-validation results indicate that model performance is broadly stable across folds, with dispersion larger for ICLN.

### Oil-shock evidence (Notebook 07)
- Following large WTI moves, XLE shows a larger average response than ICLN in the ±3-day window (see shock reaction summary by lag).
- This pattern is consistent with partial insulation of renewable equities from short-run oil shocks, while not implying causality.

### Limitations and extensions
- The feature set is WTI-centered; adding macro-financial controls (rates, VIX, inflation proxies) would help isolate oil-specific effects.
- Structural breaks (e.g., COVID-19, the Ukraine war) are not modeled explicitly and may affect stability.
- The shock definition and event window are conventional choices; sensitivity checks to thresholds and horizons would strengthen robustness.

### Final conclusion
Fossil energy equity returns (XLE) appear more exposed to oil price dynamics and therefore more predictable within this framework. Renewable energy equities (ICLN) show weaker dependence on WTI and remain harder to forecast, consistent with a broader set of underlying drivers.